In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import (year, month, dayofmonth, hour, weekofyear, 
                                   dayofweek, monotonically_increasing_id)
from pyspark.sql.types import TimestampType

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1650473450452_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# config = configparser.ConfigParser()
# config.read("dl.cfg")

# config.sections()
# os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
# os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    
    return spark

print("Creating spark session.")
spark = create_spark_session()
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Creating spark session.

In [48]:
# Paths to S3 storage buckets
input_data = "s3a://udacity-dend/"
output_data = "s3a://output-data-spark"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Process Song Data

In [49]:
# get filepath to song data file
# song_data = os.path.join(input_data, 'song_data', '*', '*', '*')
song_data = os.path.join(input_data, 'song_data', 'A', 'A', 'A')

# read song data file
df = spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
print(f"Number of entries: {df.count()}")

print("Song data schema:")
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of entries: 24
Song data schema:
root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [51]:
# extract columns to create songs table
songs_table = df.select(['song_id', 'title', 'artist_id', 'year', 'duration'])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
songs_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOAFBCP12A8C13CC7D|King Of Scurf (20...|ARTC1LV1187B9A4858|1972|301.40036|
|SOKTJDS12AF72A25E5|Drown In My Own T...|ARA23XO1187B9AF18F|   0|  192.522|
|SOEKAZG12AB018837E|I'll Slap Your Fa...|ARSVTNL1187B992A91|2001|129.85424|
|SOQPWCR12A6D4FB2A3|A Poor Recipe For...|AR73AIO1187B9AD57B|2005|118.07302|
|SOBRKGM12A8C139EF6|Welcome to the Pl...|ARXQBR11187B98A2CC|1985|821.05424|
|SORRNOC12AB017F52B|The Last Beat Of ...|ARSZ7L31187FB4E610|2004|337.81506|
|SOHKNRJ12A6701D1F8|        Drop of Rain|AR10USD1187B99F3F1|   0|189.57016|
|SOAPERH12A58A787DC|The One And Only ...|ARZ5H0P1187B98A1DD|   0|230.42567|
|SOSMJFC12A8C13DE0C|Is That All There...|AR1KTV21187B9ACD72|   0|343.87546|
|SOOVHYF12A8C134892|     I'll Be Waiting|ARCLYBR1187FB53913|1989|304.56118|
|SOERIDA12A6

In [60]:
# write songs table to parquet files partitioned by year and artist
songs_table.write.partitionBy('year', 'artist_id').parquet(os.path.join(output_data, 'songs/songs.parquet'), 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
# extract columns to create artists table
artists_table = df.select(['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude'])
artists_table = artists_table.dropDuplicates(['artist_id'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
artists_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|AR9Q9YC1187FB5609B|    Quest_ Pup_ Kevo|          New Jersey|           null|            null|
|ARA23XO1187B9AF18F|     The Smithereens|Carteret, New Jersey|       40.57885|       -74.21956|
|ARZKCQM1257509D107|          Dataphiles|                    |           null|            null|
|ARC1IHZ1187FB4E920|        Jamie Cullum|                    |           null|            null|
|ARGE7G11187FB37E05|        Cyndi Lauper|        Brooklyn, NY|           null|            null|
|AR10USD1187B99F3F1|Tweeterfriendly M...|Burlington, Ontar...|           null|            null|
|ARJNIUY12298900C91|        Adelitas Way|                    |           null|            null|
|AR73AIO1187B9AD57B|   Western Addiction

In [63]:
# write artists table to parquet files
artists_table.write.parquet(os.path.join(output_data, 'artists/artists.parquet'), 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Process Log data

In [109]:
# get filepath to log data file
log_data = os.path.join(input_data, 'log_data', '*', '*')

# read log data file
df = spark.read.json(log_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
print(f"Number of entries: {df.count()}")

print("Log data schema:")
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of entries: 8056
Log data schema:
root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [111]:
# filter by actions for song plays
df = df.where(df.page == 'NextSong')

# extract columns for users table    
users_table = df.select(
    ['userId', 'firstName', 'lastName', 'gender', 'level']
    ).dropDuplicates(['userId']).orderBy('userId')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
users_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    10|   Sylvie|    Cruz|     F| free|
|   100|    Adler| Barrera|     M| free|
|   101|   Jayden|     Fox|     M| free|
|    11|Christian|  Porter|     F| free|
|    12|   Austin| Rosales|     M| free|
|    13|      Ava|Robinson|     F| free|
|    14| Theodore|  Harris|     M| free|
|    15|     Lily|    Koch|     F| paid|
|    16|    Rylan|  George|     M| paid|
|    17| Makinley|   Jones|     F| free|
|    18|    Jacob|  Rogers|     M| free|
|    19|  Zachary|  Thomas|     M| free|
|     2|  Jizelle|Benjamin|     F| free|
|    20|    Aiden| Ramirez|     M| paid|
|    22|     Sean|  Wilson|     F| free|
|    23|   Morris| Gilmore|     M| free|
|    24|    Layla| Griffin|     F| paid|
|    25|   Jayden|  Graves|     M| paid|
|    26|     Ryan|   Smith|     M| free|
|    27|   Carlos|  Carter|     M| free|
+------+---------+--------+------+-----+
only showing top

In [69]:
# write users table to parquet files
users_table.write.parquet(os.path.join(output_data, 'users/users.parquet'), 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.fromtimestamp(int(x) / 1000.0), TimestampType())

df = df.withColumn('timestamp', get_timestamp(df.ts))

# create datetime column from original timestamp column
df = df.withColumn('datetime', get_timestamp(df.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
# extract columns to create time table
df = df.withColumn('hour', hour('timestamp'))
df = df.withColumn('day', dayofmonth('timestamp'))
df = df.withColumn('week', weekofyear('timestamp'))
df = df.withColumn('month', month('timestamp'))
df = df.withColumn('year', year('timestamp'))
df = df.withColumn('weekday', dayofweek('timestamp'))

time_table = df.select(col('datetime'),
                       col('hour'),
                       col('day'),
                       col('week'),
                       col('month'),
                       col('year'),
                       col('weekday')
                      ).dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
time_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---+----+-----+----+-------+
|            datetime|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-15 14:09:...|  14| 15|  46|   11|2018|      5|
|2018-11-15 15:24:...|  15| 15|  46|   11|2018|      5|
|2018-11-15 16:31:...|  16| 15|  46|   11|2018|      5|
|2018-11-15 19:22:...|  19| 15|  46|   11|2018|      5|
|2018-11-21 17:40:...|  17| 21|  47|   11|2018|      4|
|2018-11-14 13:30:...|  13| 14|  46|   11|2018|      4|
|2018-11-14 17:53:...|  17| 14|  46|   11|2018|      4|
|2018-11-28 14:18:...|  14| 28|  48|   11|2018|      4|
|2018-11-28 19:44:...|  19| 28|  48|   11|2018|      4|
|2018-11-05 09:55:...|   9|  5|  45|   11|2018|      2|
|2018-11-05 17:51:...|  17|  5|  45|   11|2018|      2|
|2018-11-13 10:32:...|  10| 13|  46|   11|2018|      3|
|2018-11-30 12:34:...|  12| 30|  48|   11|2018|      6|
|2018-11-30 14:33:...|  14| 30|  48|   11|2018|      6|
|2018-11-16 11:52:...|  11| 16|  46|   11|2018| 

In [74]:
# write time table to parquet files partitioned by year and month
time_table.write.partitionBy('year', 'month') \
                .parquet(os.path.join(output_data, 'time/time.parquet'), 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
# read in song data to use for songplays table
song_data = os.path.join(input_data, 'song_data', 'A', 'A', 'A')
song_df = spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
# extract columns from joined song and log datasets to create songplays table 
df = df.join(song_df, song_df.title == df.song)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [118]:
songplays_table = df.select(
        col('ts'),
        col('userId'),
        col('level'),
        col('song_id'),
        col('artist_id'),
        col('sessionId'),
        col('location'),
        col('userAgent'),
        year(df.datetime).alias('year'),
        month(df.datetime).alias('month')) \
        .withColumn('songplay_id', monotonically_increasing_id())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
songplays_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------+-----+------------------+------------------+---------+--------------------+--------------------+----+-----+-----------+
|           ts|userId|level|           song_id|         artist_id|sessionId|            location|           userAgent|year|month|songplay_id|
+-------------+------+-----+------------------+------------------+---------+--------------------+--------------------+----+-----+-----------+
|1542298745796|    97| paid|SOBLFFE12AF72AA5BA|ARJNIUY12298900C91|      605|Lansing-East Lans...|"Mozilla/5.0 (X11...|2018|   11|          0|
+-------------+------+-----+------------------+------------------+---------+--------------------+--------------------+----+-----+-----------+

In [108]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.partitionBy('year', 'month') \
    .parquet(os.path.join(output_data, 'songplays/songplays.parquet'),
             'overwrite')        

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…